In [ ]:
# Duomenys:
# https://data.gov.lt/datasets/2829/
# - Stacionaro ir ambulatoriniai ligoniai, gydyti nuo traumų ir
# apsinuodijimų. Įvertinkite patiriamų traumų, apsinuodijimų
# dažnius/pasiskirstymą, sunkumą pagal amžių/amžiaus
# grupes, lytį, paros, metų, darbo dienų/švenčių/išeiginių. Ar
# traumų pobūdis/dažnis, apsinuodijimų kiekis, pobūdis, sunkumas
# priklauso nuo užfiksuotų meteorologinių sąlygų, mėnulio fazės,
# švenčių/laisvadienių?

https://data.gov.lt/datasets/2829/data/Ligonine/ 

https://data.gov.lt/datasets/2829/data/Pacientas/ 

https://ligoniukasa.lrv.lt/uploads/ligoniukasa/documents/files/1(9).xlsx 


# Duomenys:
# https://data.gov.lt/datasets/2793/#info
# – PAGD iškvietimai; https://data.gov.lt/datasets/1667/#info
# – Policijos
# iškvietimai. Ar
# iškvietimų dažnis/kiekis priklauso nuo paros,
# metų, darbo dienų/švenčių/išeiginių, mėnulio
# fazės, švenčių/laisvadienių?
# Mėnulio
# fazės turi būti pasiekiamos per http užklausas – iš esmės
# pasinaudojate web scrapingu, kad gautumėte norimą informaciją.
# Papildoma informacija -
# https://ligoniukasa.lrv.lt/lt/veiklos-sritys/informacija-gyventojams/gydymo-ir-sveikatos-prieziuros-paslaugos/stebejimo-paslaugos/
# ;

# https://astropixels.com/ephemeris/phasescat/phasescat.html


# TLK kodų paaiškinimai yra šiame excel faile:
# https://ligoniukasa.lrv.lt/uploads/ligoniukasa/documents/files/1(9).xlsx

# Jūs turėsite sujungti tris failus:
# https://data.gov.lt/datasets/2829/data/Pacientas/
# https://data.gov.lt/datasets/2829/data/Ligonine/
# ir pateiktą excel'io failą, kad turėtumėte vientisą lentelę su reikiamais duomenimis.
# Prie jų jungsite infromaciją iš
# https://astropixels.com/ephemeris/phasescat/phasescat.html
# ARBA day.lt - bus paprasčiau:
# https://day.lt/diena/2021.04.22    <--- adreso laukelyje įrašius norimą datą, gaunama informacija apie tą dieną.

# iš kurios, naudodami web scraping'ą, išgausite reikiamą informaciją apie fazes. (requests bibliotekos užteks)

In [ ]:
# siuncia tiesiai i pythono instancija
# import requests
# r = requests.get('https://get.data.gov.lt/datasets/gov/hi/traumos_apsinuodijimai/Pacientas')
# ats = r.json()
# pandas_df = pd.DataFrame(data=ats["_data"])
# pandas_df.head()

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from datetime import datetime
import requests
from bs4 import BeautifulSoup

In [2]:
paci = pd.read_csv('../../../eismas/Pacientas.csv')
print('Nuskaityta 1')
lig = pd.read_csv('../../../eismas/Ligonine.csv')
print('Nuskaityta 2')
tlk = pd.read_excel('../../../eismas/LigoninePacientas.xlsx')
print('Nuskaityta 3')


Nuskaityta 1
Nuskaityta 2
Nuskaityta 3


In [3]:
paci.head(2)

,_type,_id,_revision,_page.next,vda_pk,paciento_id,atvejo_id,gyd_lap_pradzia,lytis,vda_gim_metai,savivaldybe,gyvenviete,gydymas,stacionaro_tipas,tlk_trauma,tlk_aplinkybe,tlk_veikla,tlk_vieta
0,datasets/gov/hi/traumos_apsinuodijimai/Pacientas,fe5742d1-21bb-4897-9716-0ea67e22e005,f68b114b-36e7-4d6e-a072-919c64f98594,NaN,000002a37f16bf4,24cdba64a645d60,062bca343d25de9,2013-01-01,Vyras,1998-01-01,Kretingos r. sav.,Nežinoma,Ambulatorinis,NaN,S42.43,NaN,NaN,NaN
1,datasets/gov/hi/traumos_apsinuodijimai/Pacientas,0252d0f8-c328-421d-a76d-32c1776650ad,50c4e3d4-752d-4243-be45-db486f717d33,NaN,000005b7496be6b,edb88dfb4887a6d,6dd099b832da48d,2022-08-01,Vyras,1973-01-01,Vilniaus r. sav.,Kaimas,Ambulatorinis,NaN,S13.6,W22,U54.08,Y92.80


In [10]:
paci.columns

Index(['_type', '_id', '_revision', '_page.next', 'vda_pk', 'paciento_id',
       'atvejo_id', 'gyd_lap_pradzia', 'lytis', 'vda_gim_metai', 'savivaldybe',
       'gyvenviete', 'gydymas', 'stacionaro_tipas', 'tlk_trauma',
       'tlk_aplinkybe', 'tlk_veikla', 'tlk_vieta'],
      dtype='object')

In [4]:
laikai = paci['gyd_lap_pradzia'].unique().tolist()
print(len(laikai))
print(laikai)

151
['2013-01-01', '2022-08-01', '2018-02-01', '2015-08-01', '2015-10-01', '2021-09-01', '2015-12-01', '2014-05-01', '2023-10-01', '2022-06-01', '2019-02-01', '2017-08-01', '2014-11-01', '2021-07-01', '2019-11-01', '2023-05-01', '2020-07-01', '2014-01-01', '2018-05-01', '2019-06-01', '2016-07-01', '2013-06-01', '2015-04-01', '2023-08-01', '2013-09-01', '2018-08-01', '2013-10-01', '2018-10-01', '2018-04-01', '2022-04-01', '2014-08-01', '2021-01-01', '2023-09-01', '2016-04-01', '2013-07-01', '2014-09-01', '2023-01-01', '2015-06-01', '2019-08-01', '2021-03-01', '2019-04-01', '2022-01-01', '2022-09-01', '2020-02-01', '2016-06-01', '2017-05-01', '2015-01-01', '2019-05-01', '2014-07-01', '2015-09-01', '2023-07-01', '2020-04-01', '2018-12-01', '2014-04-01', '2022-07-01', '2017-12-01', '2012-02-01', '2013-11-01', '2015-05-01', '2023-06-01', '2020-11-01', '2020-09-01', '2021-02-01', '2021-05-01', '2015-02-01', '2016-08-01', '2013-12-01', '2017-07-01', '2021-12-01', '2018-03-01', '2021-04-01', '

In [12]:
url = 'https://day.lt/diena/2025.02.05'

source = requests.get(url)
status = source.status_code
print(status)
# r = page.json()

200


In [20]:
# Check if the request was successful
if source.status_code == 200:
    bs = BeautifulSoup(source.text, 'html.parser')
    
    menulis = bs.find('ul', {'title':'Mėnulis'}).find('a', {'title':'Apie dabartinį mėnulį'})
    print(menulis.text)

else:
    print(f"Failed to retrieve the page. Status code: {source.status_code}")

Priešpilnis


In [23]:
fazes = {}

for laikas in laikai:
    l = laikas.replace('-', '.')
    # print(l)

    url = f'https://day.lt/diena/{l}'
    source = requests.get(url)

    if source.status_code == 200:
        bs = BeautifulSoup(source.text, 'html.parser')
        
        menulis = bs.find('ul', {'title':'Mėnulis'}).find('a', {'title':'Apie dabartinį mėnulį'})
        # print(menulis.text)

        fazes[laikas] = menulis.text

    else:
        print(f"Failed to retrieve the page. Status code: {source.status_code}")
    



2013.01.01
Pilnatis
2022.08.01
Jaunatis (pilnėja)
2018.02.01
Pilnatis
2015.08.01
Pilnatis
2015.10.01
Pilnatis
2021.09.01
Delčia
2015.12.01
Pilnatis
2014.05.01
Jaunatis
2023.10.01
Pilnatis
2022.06.01
Jaunatis
2019.02.01
Delčia
2017.08.01
Priešpilnis
2014.11.01
Priešpilnis
2021.07.01
Pilnatis
2019.11.01
Jaunatis (pilnėja)
2023.05.01
Priešpilnis (pilnėja)
2020.07.01
Priešpilnis (pilnėja)
2014.01.01
Jaunatis
2018.05.01
Pilnatis
2019.06.01
Delčia (dylantis)
2016.07.01
Delčia
2013.06.01
Delčia (dyla)
2015.04.01
Priešpilnis (pilnėja)
2023.08.01
Pilnatis
2013.09.01
Delčia
2018.08.01
Pilnatis
2013.10.01
Delčia
2018.10.01
Pilnatis
2018.04.01
Pilnatis
2022.04.01
Jaunatis
2014.08.01
Jaunatis (pilnėja)
2021.01.01
Pilnatis
2023.09.01
Pilnatis
2016.04.01
Delčia
2013.07.01
Delčia (dyla)
2014.09.01
Priešpilnis
2023.01.01
Priešpilnis
2015.06.01
Pilnatis
2019.08.01
Jaunatis
2021.03.01
Pilnatis
2019.04.01
Delčia
2022.01.01
Delčia (dylantis)
2022.09.01
Jaunatis (pilnėja)
2020.02.01
Priešpilnis
2016.06.01
D

In [36]:
print(set(fazes.values()))


{'Delčia (dylantis)', 'Jaunatis', 'Jaunatis (pilnėja)', 'Priešpilnis', 'Delčia (dyla)', 'Delčia', 'Pilnatis', 'Priešpilnis (pilnėja)'}


In [40]:
fazes1 = {key: value.replace('Jaunatis (pilnėja)', 'Jaunatis')
          .replace('Delčia (dylantis)', 'Delčia')
          .replace('Delčia (dyla)', 'Delčia')
          .replace('Priešpilnis (pilnėja)', 'Priešpilnis') for key, value in fazes.items()}
# fazes2 = {key: value.replace('Delčia (dylantis)', 'Delčia') for key, value in fazes.items()}

In [41]:
print(set(fazes1.values()))

{'Pilnatis', 'Delčia', 'Jaunatis', 'Priešpilnis'}


In [44]:
paci['iv_data'] = pd.to_datetime(paci['gyd_lap_pradzia'])
paci['moon'] = paci['iv_data'].dt.strftime('%Y-%m-%d').map(fazes1)
paci.head(2)


,_type,_id,_revision,_page.next,vda_pk,paciento_id,atvejo_id,gyd_lap_pradzia,lytis,vda_gim_metai,savivaldybe,gyvenviete,gydymas,stacionaro_tipas,tlk_trauma,tlk_aplinkybe,tlk_veikla,tlk_vieta,iv_data,moon
0,datasets/gov/hi/traumos_apsinuodijimai/Pacientas,fe5742d1-21bb-4897-9716-0ea67e22e005,f68b114b-36e7-4d6e-a072-919c64f98594,NaN,000002a37f16bf4,24cdba64a645d60,062bca343d25de9,2013-01-01,Vyras,1998-01-01,Kretingos r. sav.,Nežinoma,Ambulatorinis,NaN,S42.43,NaN,NaN,NaN,2013-01-01,Pilnatis
1,datasets/gov/hi/traumos_apsinuodijimai/Pacientas,0252d0f8-c328-421d-a76d-32c1776650ad,50c4e3d4-752d-4243-be45-db486f717d33,NaN,000005b7496be6b,edb88dfb4887a6d,6dd099b832da48d,2022-08-01,Vyras,1973-01-01,Vilniaus r. sav.,Kaimas,Ambulatorinis,NaN,S13.6,W22,U54.08,Y92.80,2022-08-01,Jaunatis


In [70]:
a = paci.iloc[0, -2]
# a = paci.loc[0:1, ['iv_data': 'moon']]
print(a)
print(a.month)
print(a.year)
print(a.day)

2013-01-01 00:00:00
1
2013
1


In [ ]:
# Function to determine season based on month
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

# Create a new 'season' column based on the month
df['season'] = df['timestamp'].dt.month.apply(get_season)